# **Lab 2: Random exploration**

This lab has a few goals:

- to get you familiar with the Python library we'll be using to simulate and explore exploration
- to introduce you to Levy walk and Brownian motion agents


### **How to go through lab**

For this lab, you will *read through the text of this notebook* and *run each code cell* in order, as well as *follow other instructions as they come up*. Try to understand what the code that you run does. *Answer any questions as Python comments where specified*.

### **What's a Levy walk? What's Brownian motion?**

* A Levy walk is is a continuing process of random movement where at each "step" of movement, a direction of and distance of movement is chosen randomly. The distance $\delta_i$ of movement at each time step $i$ is sampled from the random distribution as follows: $\delta_i = {u_i}^{-\frac{1}{\gamma}}$, where $u_i \sim N(\mu,\sigma)$ and $\gamma > 1$.
* Brownian motion is the same except for that $\gamma = 1$. Brownian motion models the behavior of physical diffusion, e.g. the movement of a an atom as it randomly bumps into other atoms. Brownian motion tends to be more local, while Levy walks tend to stay local most of the time, occasionally displaying larger "jumps".

### **Install _explorationlib_**
Colab notebooks come with many of the code libraries we will need. However, they do not come with _explorationlib_. This is a module that we will be using; it was built by Erik Peterson to support this course. Let's Install it.

In [ ]:
!pip install --upgrade git+https://github.com/coaxlab/explorationlib
!pip install --upgrade git+https://github.com/MattChanTK/gym-maze.git
!pip install celluloid # for the gifs

### **Import modules**

In [ ]:
# from the standard library
import shutil
import glob
import os
import copy
import sys

# these are common to scientific programming in python
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Computational experiments are run with 'experiment'
from explorationlib.run import experiment

# Here are some tools to select, save, and load
# data from computational experiments
from explorationlib.util import select_exp
from explorationlib.util import load
from explorationlib.util import save

# All the explorers we will play with are called
# "agents"; a bit of computer science jargon
from explorationlib.agent import Diffusion2d
from explorationlib.agent import Levy2d
#from explorationlib.agent import TruncatedLevy2d

# The environments we will simulate live in a "gym"
from explorationlib.local_gym import Field
from explorationlib.local_gym import Bounded
from explorationlib.local_gym import Grid 
from explorationlib.local_gym import uniform_targets
from explorationlib.local_gym import constant_values

# A bunch of tools for plotting and for
# movie making
from explorationlib.plot import plot_position2d
from explorationlib.plot import plot_length_hist
from explorationlib.plot import plot_length
from explorationlib.plot import plot_targets2d
from explorationlib.plot import show_gif

# A couple metrics for scoring how well, or poorly,
# an exploration experiment went.
from explorationlib.score import search_efficiency
from explorationlib.score import total_reward
from explorationlib.score import first_reward

### Configure plots

This will help the plots to be be nicer looking. We don't _have_ to do this, but _why not_?

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.greedy=True

plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = "16"

### Better autocomplete and development in our notebooks

Again, optional. If running this causes any errors, skip it.

In [ ]:
%config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2

## **Getting to know our explorers**

The code blocks below run a diffusion and levy walker for 10000 steps, in an unbounded Field. 10 different experiments are run.

### Experimental Parameters

In [ ]:
# Setup
num_steps = 10000       # how many exploration steps in space?
num_experiments = 10    # we want to run ten experiments
detection_radius = 1

num_targets = 500

min_length = 0.1
step_size = 0.1  # Should be < (2 * detection_radius)

### Create Environment

In [ ]:
# Create env as an empty field
env = Field()

### Add targets to the environment

In [ ]:
# Targets
target_boundary = (50, 50)
targets = uniform_targets(num_targets, target_boundary)
values = constant_values(targets, 1)
env.add_targets(targets, values, detection_radius=detection_radius)

### Create Agents

In [ ]:
# Create agents
diffusion = Diffusion2d(
    min_length=min_length, 
    scale=0.1, 
    detection_radius=detection_radius, 
    step_size=step_size,
)

levy = Levy2d(min_length=min_length, 
    exponent=2, 
    detection_radius=detection_radius, 
    step_size=step_size,
)

## **How do these agents work? Check the explorationlib code**

* In the left panel on Colab, click on the file icon.
* Navigate to `/usr/local/lib/python3.7/dist-packages/explorationlib/agent.py` and double click the file (or just click the link in this bullet) to open up the library's code for defining exploration agents.
* You should see the code for `agent.py` open in a panel to the right in Colab.
* Read the definition of the Agent2d class (find the spot in the code that says `class Agent2d:` and read until just before the next class, CriticGrid, is defined).
* Scroll down far and read the definition for the Levy2d class, which inherits from Agent2d.
* Scroll down and read the definition for the last class, Diffusion2d.

### Run Experiments

In [ ]:
# experiments
levy_exp = experiment(
    f"levy_1", 
    levy, 
    env, 
    num_steps=num_steps, 
    num_experiments=num_experiments,
    seed=59393,
    dump=False,
)
diffusion_exp = experiment(
    f"diffusion_1", 
    diffusion, 
    env, 
    num_steps=num_steps, 
    num_experiments=num_experiments,
    seed=59393,
    dump=False,
)

### Plot the trajectories of both agents across the 10 experiments

In [ ]:
# View size
plot_boundary = (20, 20)

# Generate 10 plots of walking
for n in range(num_experiments):
    ax = None
    ax = plot_position2d(
        select_exp(levy_exp, n),
        boundary=plot_boundary,
        label=f"Levy",
        color="blue",
        alpha=0.6,
        ax=ax,
    )
    ax = plot_position2d(
        select_exp(diffusion_exp, n),
        boundary=plot_boundary,
        label=f"Diffusion",
        title=f"Experiment {n}",
        color="brown",
        alpha=0.6,
        ax=ax,
    )

### Question 1.1
Based on these 10 example experiments please describe how similar or different the Levy and diffusion search strategies seem to be? Visually, that is. Can you tell the difference between them in _all_ 10 examples?

_Note_: Don't be afraid to change the _plot boundary_ to zoom in or out. 

In [ ]:
# Write your answer here, as a python comment

### Plot the distribution of step sizes

Create one distribution for all Levy steps, and one distribution for all Brownian steps, and plot them together. First plot the distributions, then plot them in log-log space. AKA the space in which power laws, like non-Brownian Levy step size distributions, are easiest to identify (as they should look more linear).

In [ ]:
num_experiment = 0
ax = None
ax = plot_length_hist(
    select_exp(levy_exp, num_experiment),
    loglog=False,
    bins=50,
    density=True,
    alpha=0.6,
    label="Levy",
    color="blue",
    ax=ax,
    figsize=(6,4),
)
ax = plot_length_hist(
    select_exp(diffusion_exp, num_experiment),
    loglog=False,
    bins=50,
    density=True,
    alpha=0.6,
    color="brown",
    label="Diffusion",
    ax=ax,
)
sns.despine() # Make pretty plot

- Wow, the power law is steep right!? Can you see the exponential nature of diffusion?

In [ ]:
ax = None
ax = plot_length_hist(
    select_exp(levy_exp, num_experiment),
    loglog=True,
    bins=50,
    density=True,
    alpha=0.6,
    label="Levy",
    color="blue",
    ax=ax,
    figsize=(6,4),
)
ax = plot_length_hist(
    select_exp(diffusion_exp, num_experiment),
    loglog=True,
    bins=50,
    density=True,
    alpha=0.6,
    color="brown",
    label="Diffusion",
    ax=ax,
)
sns.despine() # Make pretty plot

### Compare the efficacies of the two strategies

Note: The difference between the agents might be smaller you expect, and the difference might not even be noticeable at all without a very large sample size.

Compile the scores of the experiments

In [ ]:
# Results, names, and colors
results = [levy_exp, diffusion_exp]
names = ["Levy", "Diffusion"]
colors = ["blue", "brown"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    r = search_efficiency(res)
    scores.append(r)  

Plot the distribution of scores

In [ ]:
for (name, s, c) in zip(names, scores, colors):
    plt.hist(s, label=name, color=c, alpha=0.5, bins=20)
    plt.legend()
    plt.xlabel("Score")
    plt.tight_layout()
    sns.despine()

Plot the average score of each agent

In [ ]:
# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.6)
plt.ylabel("Score")
plt.tight_layout()
sns.despine()

This lab adapted by Jack Burgess in the fall of 2022 from materials by Matthew Clapp.